<a href="https://colab.research.google.com/github/rajni-arora/Pretraining-Bert-Transformers-models/blob/main/MLM_Training_With_Trainer_step4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook tells training process for masked-language modeling (MLM) using the HuggingFace Trainer function.

In [1]:
!pip install transformers

In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
!wget https://github.com/jamescalam/transformers/blob/main/data/text/meditations/clean.txt

--2023-09-17 17:24:30--  https://github.com/jamescalam/transformers/blob/main/data/text/meditations/clean.txt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248927 (243K) [text/plain]
Saving to: ‘clean.txt.1’

clean.txt.1         100%[===================>] 243.09K  --.-KB/s    in 0.04s   

2023-09-17 17:24:31 (5.98 MB/s) - ‘clean.txt.1’ saved [248927/248927]



In [4]:
with open('/content/clean.txt', 'r') as fp:
    text = fp.read().split('\n')

In [5]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

we create labels tensor by cloning the input_ids tensor

In [6]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [7]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [8]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

Then apply these indices to each respective row in input_ids, assigning each of the values at these indices as 103.

In [9]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

The Trainer expects a Dataset object, so we need to initialize this.

In [10]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [11]:
dataset = MeditationsDataset(inputs)


In [12]:
!pip install transformers[torch]

In [13]:
!pip install accelerate -U

In [14]:
!pip install transformers[torch,accelerate] -U

In [15]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=16,
    num_train_epochs=2
)

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset
)

And train.

In [17]:
trainer.train()

<ipython-input-10-6dccb434beb5>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


TrainOutput(global_step=2, training_loss=0.27408236265182495, metrics={'train_runtime': 27.9746, 'train_samples_per_second': 0.071, 'train_steps_per_second': 0.071, 'total_flos': 526409625600.0, 'train_loss': 0.27408236265182495, 'epoch': 2.0})